# Image Classification using Xception

In [12]:
import os
import numpy as np

import PIL.Image as Image

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
folder1 = '/content/drive/MyDrive/race0'
folder2 = '/content/drive/MyDrive/race1'
folder3 = '/content/drive/MyDrive/race2'

In [6]:
def preprocess_input(x): 
   x = np.divide(x, 255.0) 
   x = np.subtract(x, 1.0) 
   x = np.multiply(x, 2.0) 
   return x 

In [7]:
images = []
gender = []
race = []
for filename in os.listdir(folder1):
            img = Image.open(os.path.join(folder1, filename)).resize((150, 150))
            img = np.array(img)
            img = preprocess_input(img)
            images.append(img)
            gender.append(int(filename[3]))
            race.append(int(filename[5]))

In [8]:
for filename in os.listdir(folder2):
            img = Image.open(os.path.join(folder2, filename)).resize((150, 150))
            img = np.array(img)
            img = preprocess_input(img)
            images.append(img)
            gender.append(int(filename[3]))
            race.append(int(filename[5]))

In [9]:
for filename in os.listdir(folder3):
            img = Image.open(os.path.join(folder3, filename)).resize((150, 150))
            img = np.array(img)
            img = preprocess_input(img)
            images.append(img)
            gender.append(int(filename[3]))
            race.append(int(filename[5]))

In [10]:
images = np.asarray(images, dtype = 'float')
gender = np.asarray(gender, dtype = 'int')
race = np.asarray(race, dtype = 'int')

### Gender Classification

In [11]:
x_train, x_test, y_train, y_test = train_test_split(
    images,
    gender,
    test_size = 0.2,
    random_state = 42
)

In [25]:
base_model = keras.applications.xception.Xception(
    weights = 'imagenet',
    input_shape = (150, 150, 3),
    include_top = False
)

In [26]:
base_model.trainable = False

In [27]:
inputs = keras.Input(
    shape = (150, 150, 3)
)
x = base_model(inputs, training = False)
x = keras.layers.GlobalAveragePooling2D()(x)
outputs = keras.layers.Dense(1, activation = 'sigmoid')(x)
model = keras.Model(inputs, outputs)

In [28]:
model.compile(
    optimizer = 'adam',
    loss = 'BinaryCrossentropy',
    metrics = ['accuracy']
)

In [29]:
results = model.fit(x_train, 
          y_train, 
          epochs = 8,
          batch_size = 8,
          validation_split = 0.2
)

Epoch 1/8
240/240 [==============================] - 24s 84ms/step - loss: 0.4195 - accuracy: 0.8058 - val_loss: 0.3681 - val_accuracy: 0.8458
Epoch 2/8
240/240 [==============================] - 17s 71ms/step - loss: 0.3399 - accuracy: 0.8544 - val_loss: 0.3633 - val_accuracy: 0.8542
Epoch 3/8
240/240 [==============================] - 17s 71ms/step - loss: 0.3137 - accuracy: 0.8653 - val_loss: 0.3750 - val_accuracy: 0.8521
Epoch 4/8
240/240 [==============================] - 19s 79ms/step - loss: 0.2955 - accuracy: 0.8763 - val_loss: 0.3548 - val_accuracy: 0.8542
Epoch 5/8
240/240 [==============================] - 17s 71ms/step - loss: 0.2785 - accuracy: 0.8847 - val_loss: 0.3643 - val_accuracy: 0.8458
Epoch 6/8
240/240 [==============================] - 19s 81ms/step - loss: 0.2588 - accuracy: 0.8951 - val_loss: 0.3633 - val_accuracy: 0.8458
Epoch 7/8
240/240 [==============================] - 19s 80ms/step - loss: 0.2522 - accuracy: 0.8987 - val_loss: 0.3795 - val_accuracy: 0.8562

In [30]:
score = model.evaluate(x_test, y_test)
test_accuracy = score[1]
print(test_accuracy)

19/19 [==============================] - 6s 189ms/step - loss: 0.3129 - accuracy: 0.8781
0.8781301975250244


### Race Classification

In [93]:
x_train, x_test, y_train, y_test = train_test_split(
    images,
    race,
    test_size = 0.2,
    random_state = 42
)

In [94]:
base_model = keras.applications.xception.Xception(
    weights = 'imagenet',
    input_shape = (150, 150, 3),
    include_top = False
)

In [95]:
base_model.trainable = False

In [97]:
inputs = keras.Input(
    shape = (150, 150, 3)
)
x = base_model(inputs, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dense(1000, activation = 'relu')(x)
x = keras.layers.Dropout(0.3)(x)
x = keras.layers.Dense(250, activation = 'relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(3, activation = 'softmax')(x)
model = keras.Model(inputs, outputs)

In [98]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [99]:
results = model.fit(x_train, 
          y_train, 
          epochs = 10,
          batch_size = 8,
          validation_split = 0.15
)

Epoch 1/10
255/255 [==============================] - 22s 73ms/step - loss: 1.0692 - accuracy: 0.4936 - val_loss: 0.8864 - val_accuracy: 0.5944
Epoch 2/10
255/255 [==============================] - 17s 67ms/step - loss: 0.8786 - accuracy: 0.6007 - val_loss: 0.8918 - val_accuracy: 0.5639
Epoch 3/10
255/255 [==============================] - 17s 67ms/step - loss: 0.8168 - accuracy: 0.6424 - val_loss: 0.8526 - val_accuracy: 0.6194
Epoch 4/10
255/255 [==============================] - 17s 67ms/step - loss: 0.7536 - accuracy: 0.6621 - val_loss: 0.8921 - val_accuracy: 0.6444
Epoch 5/10
255/255 [==============================] - 17s 67ms/step - loss: 0.6738 - accuracy: 0.7033 - val_loss: 0.8437 - val_accuracy: 0.6528
Epoch 6/10
255/255 [==============================] - 17s 67ms/step - loss: 0.6302 - accuracy: 0.7195 - val_loss: 0.8525 - val_accuracy: 0.6389
Epoch 7/10
255/255 [==============================] - 17s 68ms/step - loss: 0.5609 - accuracy: 0.7662 - val_loss: 1.0221 - val_accuracy:

In [100]:
score = model.evaluate(x_test, y_test)
test_accuracy = score[1]
print(test_accuracy)

19/19 [==============================] - 4s 142ms/step - loss: 0.9935 - accuracy: 0.6528
0.6527546048164368
